<a href="https://colab.research.google.com/github/shameer-phy/GenAI/blob/main/Sentence%20Classification/Sentence_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Reading the dataset into a dataframe

In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/clairett/pytorch-sentiment-classification/master/data/SST2/train.tsv",
                 delimiter='\t', header=None, nrows=3000)
df.columns = ["sentence", "label"]
df.head()

,sentence,label
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


#Import pretrained DistilBERT model and tokenizer

In [3]:
import torch
import transformers as T


In [4]:
model_class, tokenizer_class, model_weights = (T.DistilBertModel, T.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(model_weights)
model = model_class.from_pretrained(model_weights)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
tokenized_df = df['sentence'].apply(lambda row: tokenizer.encode(row, add_special_tokens=True))
tokenized_df.head()

,sentence
0,"[101, 1037, 18385, 1010, 6057, 1998, 2633, 182..."
1,"[101, 4593, 2128, 27241, 23931, 2013, 1996, 62..."
2,"[101, 2027, 3653, 23545, 2037, 4378, 24185, 10..."
3,"[101, 2023, 2003, 1037, 17453, 14726, 19379, 1..."
4,"[101, 5655, 6262, 1005, 1055, 12075, 2571, 376..."


#Padding to get uniform length

In [6]:
max_row_len = tokenized_df.apply(lambda row: len(row)).max()

In [7]:
def pad(r):
  for i in range(max_row_len - len(r)):
    r.append(0)
  return r

In [8]:
padding_df = tokenized_df.apply(lambda row: pad(row) if len(row) < max_row_len else row )

#Converting dataframe into a tensor and passing input to DistilBert model

In [9]:
#import torch
#torch.cuda.is_available()

torch.cuda.empty_cache()

#import os

#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [10]:
input_ids = torch.tensor(padding_df)

In [11]:
input_ids = input_ids.to('cuda')
model.to('cuda')

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [12]:
input_ids.shape

torch.Size([3000, 66])

In [13]:
with torch.no_grad():
    last_hidden_states = model(input_ids)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [14]:
model.to('cpu')
torch.cuda.empty_cache()

In [28]:
last_hidden_states[0].shape
last_hidden_states[0][:,0,:]

torch.Size([3000, 66, 768])

In [30]:
features = last_hidden_states[0][:,0,:].cpu().numpy()

In [34]:
features.shape

(3000, 768)

In [35]:
features

array([[-0.23809835, -0.43509537,  0.15847825, ...,  0.07754557,
         0.50675666,  0.08491781],
       [-0.09663326, -0.30727768,  0.28786814, ...,  0.0148513 ,
         0.46803465,  0.08588845],
       [-0.33705294, -0.133249  ,  0.09216366, ...,  0.01679165,
         0.63400185,  0.16028602],
       ...,
       [ 0.2568323 , -0.24124385,  0.259253  , ..., -0.05531594,
         0.63383734, -0.01766262],
       [-0.2599087 , -0.5266708 ,  0.24140108, ..., -0.01122085,
         0.47221708,  0.09805787],
       [-0.27983603, -0.22648506,  0.23174943, ...,  0.05789895,
         0.5141484 , -0.09972286]], dtype=float32)

#Above dataframe will serve as the input for logistic regression model that will help us classify the sentence

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [37]:
train_features, test_features, train_labels, test_labels = train_test_split(features, df['label'])

In [39]:
classifier = LogisticRegression()
classifier.fit(train_features, train_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [40]:
classifier.score(test_features, test_labels)

0.8333333333333334